Imports + config

In [23]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_poisson_deviance

DATA_PATH = "influxDB Backup/saw_wide_maschineRunning_true_joined.csv"
TARGET = "currentRMS"
DROP_COLS = ["Experiment No","Piece No","time","Material","temperatureRMS","machineRunning","microphoneRMS","vibrationRMS"]
RANDOM_STATE = 42


Train ML model (minimal)

In [24]:
df = pd.read_csv(DATA_PATH)
df.columns = df.columns.str.strip()

y = pd.to_numeric(df[TARGET], errors="coerce")
X_raw = df.drop(columns=[TARGET] + [c for c in DROP_COLS if c in df.columns], errors="ignore")

m = y.notna()
X_raw, y = X_raw.loc[m], y.loc[m]
if (y < 0).any():
    raise ValueError("Poisson needs y >= 0")

# Save raw feature columns for the optimizer input format
RAW_FEATURE_COLS = X_raw.columns.tolist()

X = pd.get_dummies(X_raw, drop_first=False).apply(pd.to_numeric, errors="coerce").fillna(0)

Xtr, Xva, ytr, yva = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
Xtr, Xva = Xtr.align(Xva, axis=1, join="left", fill_value=0)

# Save final feature space (after one-hot)
FEATURE_COLS = Xtr.columns.tolist()

model = xgb.XGBRegressor(
    objective="count:poisson",
    tree_method="hist",
    eval_metric="poisson-nloglik",
    random_state=RANDOM_STATE,
    n_jobs=-1
)

grid = {
    "n_estimators": [400, 800],
    "learning_rate": [0.05, 0.1],
    "max_depth": [3, 5],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
}

gs = GridSearchCV(model, grid, scoring="neg_mean_poisson_deviance", cv=5, n_jobs=-1, verbose=1)
gs.fit(Xtr, ytr)

best = gs.best_estimator_

p_tr, p_va = best.predict(Xtr), best.predict(Xva)

print("best:", gs.best_params_)
print("train rmse/r2:", np.sqrt(mean_squared_error(ytr, p_tr)), r2_score(ytr, p_tr))
print("valid rmse/r2:", np.sqrt(mean_squared_error(yva, p_va)), r2_score(yva, p_va))
print("valid poisson dev:", mean_poisson_deviance(yva, np.clip(p_va, 1e-12, None)))


Fitting 5 folds for each of 32 candidates, totalling 160 fits
best: {'colsample_bytree': 1.0, 'learning_rate': 0.05, 'max_depth': 5, 'n_estimators': 400, 'subsample': 0.8}
train rmse/r2: 0.5460982190494228 0.7050217766526432
valid rmse/r2: 0.5299218260664966 0.7362900171680504
valid poisson dev: 0.1412275876938728


Optimization (simple offset + SciPy minimize)

In [25]:
from scipy.optimize import minimize
import pandas as pd
import numpy as np

TARGET_CURRENT = 1.7

# Set these to your real column names + limits
BAND_COL = "bandVelocityActual"
FEED_COL = "feedRateActual"
BAND_BOUNDS = (5.0, 60.0)
FEED_BOUNDS = (50.0, 800.0)

def _X(row: dict) -> pd.DataFrame:
    # Same preprocessing as training: get_dummies + align to training feature space
    df1 = pd.DataFrame([row]).reindex(columns=RAW_FEATURE_COLS)
    x1 = pd.get_dummies(df1, drop_first=False).apply(pd.to_numeric, errors="coerce").fillna(0)
    return x1.reindex(columns=FEATURE_COLS, fill_value=0)

def suggest_settings(row: dict, measured_current: float, target_current: float = TARGET_CURRENT) -> dict:
    # 1) Use model to predict expected current consumption: c_exp = f(x)
    c_exp_cur = float(best.predict(_X(row))[0])

    # 2) Calculate offset between model prediction and actual current: offset = c_act - c_exp
    offset = float(measured_current) - c_exp_cur

    # 3+4) Minimize error by adjusting band/feed:
    # error = c_tar - (c_exp + offset)  -> minimize error^2
    b0, f0 = float(row[BAND_COL]), float(row[FEED_COL])

    def obj(z):
        b, f = float(z[0]), float(z[1])
        r = dict(row)
        r[BAND_COL] = b
        r[FEED_COL] = f
        c_exp = float(best.predict(_X(r))[0])
        err = target_current - (c_exp + offset)
        return float(err * err)

    res = minimize(obj, x0=[b0, f0], method="L-BFGS-B", bounds=[BAND_BOUNDS, FEED_BOUNDS])

    b_opt, f_opt = float(res.x[0]), float(res.x[1])
    r = dict(row); r[BAND_COL] = b_opt; r[FEED_COL] = f_opt
    c_exp_opt = float(best.predict(_X(r))[0])
    c_est_opt = c_exp_opt + offset

    return {
        "band_opt": b_opt,
        "feed_opt": f_opt,
        "offset": offset,
        "c_exp_current": c_exp_cur,
        "c_est_opt": c_est_opt,
        "ok": bool(res.success),
    }


Test

In [26]:
row = {
    BAND_COL: 39.0,
    FEED_COL: 200.0,
    "Material thickness (mm)": 10.0,  # must match your real column name
}

measured_current = 4.5  # from sensor

out = suggest_settings(row, measured_current)
print(out)  # operator applies band_opt / feed_opt


{'band_opt': 39.0, 'feed_opt': 200.0, 'offset': 3.534230053424835, 'c_exp_current': 0.9657699465751648, 'c_est_opt': 4.5, 'ok': True}
